<a href="https://colab.research.google.com/github/Mehedihasan097/Financial-Engineering/blob/main/Derivative_Pricing_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

8. **Pricing an ATM American Call and Put Using a Binomial Tree**

Here, we want to find the price of ATM (At-the-Money) options, which means the option's exercise price is equal to the current price of the asset. This method helps ensure we understand the baseline value of the option when it's at its most basic condition.  

Using many steps in a binomial tree gives a more accurate estimate of the option's value. More steps let us “zoom in” on the range of possible future prices, providing a finer understanding of the potential price movements. For example, choosing 200 steps means we're considering many small price movements, making our estimate precise and realistic.  

**Pricing Process**:

1.   We start by calculating the possible future prices at each step (each up or down movement).
2.   For each final price at the expiration date, we check whether exercising the option is profitable. For a call option, we calculate the profit if the price exceeds the strike price; for a put option, we calculate profit if it's below.
3.  Moving backward from the expiration date, we apply the option values at each node, combining possible outcomes to estimate today's price.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [21]:
# Parameters
S0 = 100      # Initial stock price
K = 100       # Strike price, ATM implies K = S0
r = 0.05      # Risk-free rate (5%)
sigma = 0.20  # Volatility (20%)
T = 3 / 12    # Maturity in years (3 months)
steps = 100   # Number of steps in the binomial tree

def calculate_parameters(S0, K, r, sigma, T, steps):
    """Calculate parameters for the binomial tree."""
    dt = T / steps
    u = np.exp(sigma * np.sqrt(dt))  # Up factor
    d = 1 / u                        # Down factor
    q = (np.exp(r * dt) - d) / (u - d)  # Risk-neutral probability
    return dt, u, d, q

def initialize_stock_prices(S0, u, d, steps):
    """Initialize a matrix for stock prices at each node in the binomial tree."""
    ST_matrix = np.zeros((steps + 1, steps + 1))
    for j in range(steps + 1):
        for i in range(j + 1):
            ST_matrix[i, j] = S0 * (u ** (j - i)) * (d ** i)  # Calculate stock price at each node
    return ST_matrix

def initialize_option_payoffs(ST_matrix, K, steps):
    """Initialize option payoffs at maturity for call and put options."""
    call_matrix = np.maximum(0, ST_matrix[:, steps] - K)
    put_matrix = np.maximum(0, K - ST_matrix[:, steps])
    return call_matrix, put_matrix

def build_binomial_tree(call_matrix, put_matrix, ST_matrix, r, q, K, dt, steps):
    """Build the binomial tree for American option prices."""
    for j in range(steps - 1, -1, -1):
        for i in range(j + 1):
            # Continuation value
            call_continuation = np.exp(-r * dt) * (q * call_matrix[i] + (1 - q) * call_matrix[i + 1])
            put_continuation = np.exp(-r * dt) * (q * put_matrix[i] + (1 - q) * put_matrix[i + 1])

            # Early exercise for American options
            call_matrix[i] = max(call_continuation, ST_matrix[i, j] - K)
            put_matrix[i] = max(put_continuation, K - ST_matrix[i, j])

    # Return the root values (t=0)
    return call_matrix[0], put_matrix[0]

def american_option_pricing(S0, K, r, sigma, T, steps):
    """Main function to calculate American call and put option prices using a binomial tree."""
    # Step 1: Calculate parameters
    dt, u, d, q = calculate_parameters(S0, K, r, sigma, T, steps)

    # Step 2: Initialize stock prices across the binomial tree
    ST_matrix = initialize_stock_prices(S0, u, d, steps)

    # Step 3: Initialize option payoffs at maturity
    call_matrix, put_matrix = initialize_option_payoffs(ST_matrix, K, steps)

    # Step 4: Expand option payoffs across the binomial tree
    american_call_price, american_put_price = build_binomial_tree(
        call_matrix, put_matrix, ST_matrix, r, q, K, dt, steps
    )

    return american_call_price, american_put_price, ST_matrix

# Run the model
american_call_price, american_put_price, ST_matrix = american_option_pricing(S0, K, r, sigma, T, steps)

print("American Call Option Price (ATM) at t=0:", round(american_call_price, 2))
print("American Put Option Price (ATM) at t=0:", round(american_put_price, 2))

# Convert the underlying asset price matrix to a DataFrame for better readability
underlying_asset_df = pd.DataFrame(ST_matrix)
print("\nBinomial Tree for Stock Prices:")
print(underlying_asset_df)

# Display the Underlying Asset Price Matrix (Binomial Tree for Stock Prices)
print("\nUnderlying Asset Price Matrix:")
print(underlying_asset_df)


American Call Option Price (ATM) at t=0: 4.61
American Put Option Price (ATM) at t=0: 3.47

Binomial Tree for Stock Prices:
       0           1           2           3           4           5    \
0    100.0  101.005017  102.020134  103.045453  104.081077  105.127110   
1      0.0   99.004983  100.000000  101.005017  102.020134  103.045453   
2      0.0    0.000000   98.019867   99.004983  100.000000  101.005017   
3      0.0    0.000000    0.000000   97.044553   98.019867   99.004983   
4      0.0    0.000000    0.000000    0.000000   96.078944   97.044553   
..     ...         ...         ...         ...         ...         ...   
96     0.0    0.000000    0.000000    0.000000    0.000000    0.000000   
97     0.0    0.000000    0.000000    0.000000    0.000000    0.000000   
98     0.0    0.000000    0.000000    0.000000    0.000000    0.000000   
99     0.0    0.000000    0.000000    0.000000    0.000000    0.000000   
100    0.0    0.000000    0.000000    0.000000    0.000000    

**Optimal number of steps N in binomial tree**

Finding the optimal number of steps N in a binomial tree is a key challenge in option pricing, especially for American options that allow early exercise. In theory, increasing N should yield more accurate results as it better approximates the continuous nature of stock price movements and the flexibility of early exercise. However, there are trade-offs in terms of computational efficiency, so the goal is to find the smallest N that provides a reliable and accurate price estimate.

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Parameters
S0 = 100      # Initial stock price
K = 100       # Strike price, ATM implies K = S0
r = 0.05      # Risk-free rate (5%)
sigma = 0.20  # Volatility (20%)
T = 3 / 12    # Maturity in years (3 months)
convergence_threshold = 0.01  # Convergence threshold for option price

def calculate_parameters(S0, K, r, sigma, T, steps):
    dt = T / steps
    u = np.exp(sigma * np.sqrt(dt))  # Up factor
    d = 1 / u                        # Down factor
    q = (np.exp(r * dt) - d) / (u - d)  # Risk-neutral probability
    return dt, u, d, q

def initialize_stock_prices(S0, u, d, steps):
    ST_matrix = np.zeros((steps + 1, steps + 1))
    for j in range(steps + 1):
        for i in range(j + 1):
            ST_matrix[i, j] = S0 * (u ** (j - i)) * (d ** i)
    return ST_matrix

def initialize_option_payoffs(ST_matrix, K, steps):
    call_matrix = np.maximum(0, ST_matrix[:, steps] - K)
    put_matrix = np.maximum(0, K - ST_matrix[:, steps])
    return call_matrix, put_matrix

def build_binomial_tree(call_matrix, put_matrix, ST_matrix, r, q, K, dt, steps):
    for j in range(steps - 1, -1, -1):
        for i in range(j + 1):
            call_continuation = np.exp(-r * dt) * (q * call_matrix[i] + (1 - q) * call_matrix[i + 1])
            put_continuation = np.exp(-r * dt) * (q * put_matrix[i] + (1 - q) * put_matrix[i + 1])
            call_matrix[i] = max(call_continuation, ST_matrix[i, j] - K)
            put_matrix[i] = max(put_continuation, K - ST_matrix[i, j])
    return call_matrix[0], put_matrix[0]

def american_option_pricing(S0, K, r, sigma, T, steps):
    dt, u, d, q = calculate_parameters(S0, K, r, sigma, T, steps)
    ST_matrix = initialize_stock_prices(S0, u, d, steps)
    call_matrix, put_matrix = initialize_option_payoffs(ST_matrix, K, steps)
    american_call_price, american_put_price = build_binomial_tree(
        call_matrix, put_matrix, ST_matrix, r, q, K, dt, steps
    )
    return american_call_price, american_put_price

def find_optimal_N(S0, K, r, sigma, T, initial_steps=10, max_steps=500):
    N = initial_steps
    previous_call_price, previous_put_price = american_option_pricing(S0, K, r, sigma, T, N)

    while N <= max_steps:
        N += 10  # Increment steps by 10 to reduce computation
        current_call_price, current_put_price = american_option_pricing(S0, K, r, sigma, T, N)

        # Check for convergence on both call and put prices
        if abs(current_call_price - previous_call_price) < convergence_threshold and \
           abs(current_put_price - previous_put_price) < convergence_threshold:
            break

        # Update previous prices for next comparison
        previous_call_price, previous_put_price = current_call_price, current_put_price

    return N, current_call_price, current_put_price

# Run the optimal N calculation
optimal_N, optimal_call_price, optimal_put_price = find_optimal_N(S0, K, r, sigma, T)

print("Optimal Number of Steps (N):", optimal_N)
print("American Call Option Price (ATM) at optimal N:", round(optimal_call_price, 2))
print("American Put Option Price (ATM) at optimal N:", round(optimal_put_price, 2))


Optimal Number of Steps (N): 40
American Call Option Price (ATM) at optimal N: 4.59
American Put Option Price (ATM) at optimal N: 3.47


9. **Computing Delta for American Call and Put Options**

Delta is one of the most important Greeks in option pricing. It measures the sensitivity of the option price to small changes in the price of the underlying asset.

For a call option, delta indicates how much the option price will increase when the underlying asset's price rises by a small amount. For an American call option, the delta is typically positive because as the stock price rises, the value of a call option also increases. This positive delta signifies that call options gain value when the stock price increases.
For a put option, delta indicates how much the option price will decrease when the underlying asset's price rises by a small amount.For an American put option, the delta is usually negative because as the stock price rises, the value of a put option decreases. This negative delta reflects that put options lose value when the stock price goes up.

The difference in signs between call and put Delta reflects the options opposite reactions to changes in the stock price. Delta provides insight into an option's directionality and can serve as a proxy for the option's probability of expiring in-the-money.

A positive Delta aligns with the notion that call options benefit from a rising underlying price whereas a negative Delta for puts represents their tendency to benefit from a decline in the underlying asset's price.

In [27]:
import numpy as np

# Parameters
S0 = 100      # Initial stock price
K = 100       # Strike price
r = 0.05      # Risk-free rate (5%)
sigma = 0.20  # Volatility (20%)
T = 3 / 12    # Maturity in years (3 months)
steps = 100   # Number of steps in the binomial tree
delta_S = 1   # Small change in stock price to calculate delta

def calculate_parameters(S, K, r, sigma, T, steps):
    dt = T / steps
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    q = (np.exp(r * dt) - d) / (u - d)
    return dt, u, d, q

def initialize_stock_prices(S, u, d, steps):
    ST_matrix = np.zeros((steps + 1, steps + 1))
    for j in range(steps + 1):
        for i in range(j + 1):
            ST_matrix[i, j] = S * (u ** (j - i)) * (d ** i)
    return ST_matrix

def initialize_option_payoffs(ST_matrix, K, steps):
    call_matrix = np.maximum(0, ST_matrix[:, steps] - K)
    put_matrix = np.maximum(0, K - ST_matrix[:, steps])
    return call_matrix, put_matrix

def build_binomial_tree(call_matrix, put_matrix, ST_matrix, r, q, K, dt, steps):
    for j in range(steps - 1, -1, -1):
        for i in range(j + 1):
            call_continuation = np.exp(-r * dt) * (q * call_matrix[i] + (1 - q) * call_matrix[i + 1])
            put_continuation = np.exp(-r * dt) * (q * put_matrix[i] + (1 - q) * put_matrix[i + 1])
            call_matrix[i] = max(call_continuation, ST_matrix[i, j] - K)
            put_matrix[i] = max(put_continuation, K - ST_matrix[i, j])
    return call_matrix[0], put_matrix[0]

def american_option_pricing(S, K, r, sigma, T, steps):
    dt, u, d, q = calculate_parameters(S, K, r, sigma, T, steps)
    ST_matrix = initialize_stock_prices(S, u, d, steps)
    call_matrix, put_matrix = initialize_option_payoffs(ST_matrix, K, steps)
    american_call_price, american_put_price = build_binomial_tree(
        call_matrix, put_matrix, ST_matrix, r, q, K, dt, steps
    )
    return american_call_price, american_put_price

# Calculate Delta using finite differences
# Price options with S0 + delta_S
call_up, put_up = american_option_pricing(S0 + delta_S, K, r, sigma, T, steps)

# Price options with S0 - delta_S
call_down, put_down = american_option_pricing(S0 - delta_S, K, r, sigma, T, steps)

# Delta for Call and Put options
delta_call = (call_up - call_down) / (2 * delta_S)
delta_put = (put_up - put_down) / (2 * delta_S)

print("Delta for American Call Option:", round(delta_call, 4))
print("Delta for American Put Option:", round(delta_put, 4))


Delta for American Call Option: 0.5693
Delta for American Put Option: -0.4496


10. **Sensitivity to Volatility (Vega)**

Vega is positive for both call and put options, as an increase in volatility generally increases the potential for the underlying asset price to move favorably in either direction. Higher volatility leads to a wider range of potential outcomes, which increases the option's value due to the greater probability of reaching a profitable exercise condition.

Higher volatility increases the likelihood of the asset price moving above the strike price, thus benefiting call options.

Similarly, higher volatility increases the chances of the asset price moving below the strike price, which is favorable for put options.

In [30]:
import numpy as np

# Parameters
S0 = 100      # Initial stock price
K = 100       # Strike price
r = 0.05      # Risk-free rate (5%)
sigma = 0.20  # Original volatility (20%)
T = 3 / 12    # Maturity in years (3 months)
steps = 100   # Number of steps in the binomial tree
delta_sigma = 0.05  # Change in volatility to calculate vega (5%)

def calculate_parameters(S, K, r, sigma, T, steps):
    dt = T / steps
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    q = (np.exp(r * dt) - d) / (u - d)
    return dt, u, d, q

def initialize_stock_prices(S, u, d, steps):
    ST_matrix = np.zeros((steps + 1, steps + 1))
    for j in range(steps + 1):
        for i in range(j + 1):
            ST_matrix[i, j] = S * (u ** (j - i)) * (d ** i)
    return ST_matrix

def initialize_option_payoffs(ST_matrix, K, steps):
    call_matrix = np.maximum(0, ST_matrix[:, steps] - K)
    put_matrix = np.maximum(0, K - ST_matrix[:, steps])
    return call_matrix, put_matrix

def build_binomial_tree(call_matrix, put_matrix, ST_matrix, r, q, K, dt, steps):
    for j in range(steps - 1, -1, -1):
        for i in range(j + 1):
            call_continuation = np.exp(-r * dt) * (q * call_matrix[i] + (1 - q) * call_matrix[i + 1])
            put_continuation = np.exp(-r * dt) * (q * put_matrix[i] + (1 - q) * put_matrix[i + 1])
            call_matrix[i] = max(call_continuation, ST_matrix[i, j] - K)
            put_matrix[i] = max(put_continuation, K - ST_matrix[i, j])
    return call_matrix[0], put_matrix[0]

def american_option_pricing(S, K, r, sigma, T, steps):
    dt, u, d, q = calculate_parameters(S, K, r, sigma, T, steps)
    ST_matrix = initialize_stock_prices(S, u, d, steps)
    call_matrix, put_matrix = initialize_option_payoffs(ST_matrix, K, steps)
    american_call_price, american_put_price = build_binomial_tree(
        call_matrix, put_matrix, ST_matrix, r, q, K, dt, steps
    )
    return american_call_price, american_put_price

# Original option prices with initial volatility
call_original, put_original = american_option_pricing(S0, K, r, sigma, T, steps)

# Option prices with increased volatility (sigma + delta_sigma)
call_increased_vol, put_increased_vol = american_option_pricing(S0, K, r, sigma + delta_sigma, T, steps)

# Calculate Vega for Call and Put options
vega_call = (call_increased_vol - call_original) / delta_sigma
vega_put = (put_increased_vol - put_original) / delta_sigma

# Display original and increased volatility option prices along with Vega values
print("=== American Call Option ===")
print("Original Call Price (20% volatility):", round(call_original, 4))
print("Increased Call Price (25% volatility):", round(call_increased_vol, 4))
print("Vega (sensitivity to volatility):", round(vega_call, 4))

print("\n=== American Put Option ===")
print("Original Put Price (20% volatility):", round(put_original, 4))
print("Increased Put Price (25% volatility):", round(put_increased_vol, 4))
print("Vega (sensitivity to volatility):", round(vega_put, 4))

# Comparison Summary
if vega_call > vega_put:
    print("\nThe American call option shows a higher sensitivity to changes in volatility than the put option.")
else:
    print("\nThe American put option shows a higher sensitivity to changes in volatility than the call option.")

=== American Call Option ===
Original Call Price (20% volatility): 4.605
Increased Call Price (25% volatility): 5.586
Vega (sensitivity to volatility): 19.6189

=== American Put Option ===
Original Put Price (20% volatility): 3.4746
Increased Put Price (25% volatility): 4.4528
Vega (sensitivity to volatility): 19.5652

The American call option shows a higher sensitivity to changes in volatility than the put option.


15. **European call options using a trinomial tree with varying moneyness levels**

**Trinomial Tree**: A trinomial tree is an extension of the binomial tree, allowing for three possible movements at each node: up, down, and staying the same.

**Moneyness and Strike Prices**: Moneyness measures the option's intrinsic value in relation to the underlying asset price.

*   Deep Out-of-the-Money : The strike is much higher than the current price
*   Out-of-the-Money : Strike is slightly higher than the current price.
*   At-the-Money: Strike is approximately equal to the current price.
*   In-the-Money : Strike is slightly lower than the current price.
*   Deep In-the-Money : Strike is much lower than the current price.


**Expected Trend**: We expect that as the strike price decreases (moving from Deep OTM to Deep ITM), the option prices will increase because the option has a higher probability of expiring in the money.







In [31]:
import numpy as np

# Parameters
S0 = 100      # Initial stock price
r = 0.05      # Risk-free rate (5%)
sigma = 0.20  # Volatility (20%)
T = 3 / 12    # Maturity in years (3 months)
steps = 100   # Number of steps in the trinomial tree

# Strike prices based on moneyness
strike_prices = {
    "Deep OTM": S0 * 1.1,
    "OTM": S0 * 1.05,
    "ATM": S0 * 1.0,
    "ITM": S0 * 0.95,
    "Deep ITM": S0 * 0.9
}

def calculate_trinomial_parameters(S0, r, sigma, T, steps):
    dt = T / steps
    u = np.exp(sigma * np.sqrt(2 * dt))   # Up factor
    d = 1 / u                             # Down factor
    m = 1                                 # No change factor (middle node)

    # Risk-neutral probabilities
    pu = ((np.exp(r * dt / 2) - np.exp(-sigma * np.sqrt(dt / 2))) / (np.exp(sigma * np.sqrt(dt / 2)) - np.exp(-sigma * np.sqrt(dt / 2))))**2
    pd = ((np.exp(sigma * np.sqrt(dt / 2)) - np.exp(r * dt / 2)) / (np.exp(sigma * np.sqrt(dt / 2)) - np.exp(-sigma * np.sqrt(dt / 2))))**2
    pm = 1 - pu - pd

    return dt, u, d, m, pu, pm, pd

def trinomial_tree_option_price(S0, K, r, sigma, T, steps):
    dt, u, d, m, pu, pm, pd = calculate_trinomial_parameters(S0, r, sigma, T, steps)

    # Initialize asset prices at maturity
    ST = np.zeros((2 * steps + 1,))
    for i in range(2 * steps + 1):
        ST[i] = S0 * (u ** max(0, steps - i)) * (d ** max(0, i - steps))

    # Option values at maturity
    call_values = np.maximum(ST - K, 0)

    # Step back through the tree
    for j in range(steps - 1, -1, -1):
        for i in range(2 * j + 1):
            call_values[i] = np.exp(-r * dt) * (pu * call_values[i] + pm * call_values[i + 1] + pd * call_values[i + 2])

    return call_values[0]

# Price call options for each strike
option_prices = {}
for moneyness, K in strike_prices.items():
    price = trinomial_tree_option_price(S0, K, r, sigma, T, steps)
    option_prices[moneyness] = price
    print(f"European Call Option Price for {moneyness} (Strike={K}): {round(price, 4)}")

# Display trend in option prices
print("\nTrend in option prices with respect to moneyness:")
for moneyness, price in option_prices.items():
    print(f"{moneyness}: {round(price, 4)}")


European Call Option Price for Deep OTM (Strike=110.00000000000001): 1.1918
European Call Option Price for OTM (Strike=105.0): 2.4819
European Call Option Price for ATM (Strike=100.0): 4.61
European Call Option Price for ITM (Strike=95.0): 7.7176
European Call Option Price for Deep ITM (Strike=90.0): 11.6717

Trend in option prices with respect to moneyness:
Deep OTM: 1.1918
OTM: 2.4819
ATM: 4.61
ITM: 7.7176
Deep ITM: 11.6717


16. **European call options using a trinomial tree with varying moneyness levels**

In [2]:
import numpy as np

# Parameters
S0 = 100      # Initial stock price
r = 0.05      # Risk-free rate (5%)
sigma = 0.20  # Volatility (20%)
T = 3 / 12    # Maturity in years (3 months)
steps = 100   # Number of steps in the trinomial tree

# Strike prices based on moneyness
strike_prices = {
    "Deep OTM": S0 * 1.1,
    "OTM": S0 * 1.05,
    "ATM": S0 * 1.0,
    "ITM": S0 * 0.95,
    "Deep ITM": S0 * 0.9
}

def calculate_trinomial_parameters(S0, r, sigma, T, steps):
    dt = T / steps
    u = np.exp(sigma * np.sqrt(2 * dt))   # Up factor
    d = 1 / u                             # Down factor
    m = 1                                 # No change factor (middle node)

    # Risk-neutral probabilities
    pu = ((np.exp(r * dt / 2) - np.exp(-sigma * np.sqrt(dt / 2))) / (np.exp(sigma * np.sqrt(dt / 2)) - np.exp(-sigma * np.sqrt(dt / 2))))**2
    pd = ((np.exp(sigma * np.sqrt(dt / 2)) - np.exp(r * dt / 2)) / (np.exp(sigma * np.sqrt(dt / 2)) - np.exp(-sigma * np.sqrt(dt / 2))))**2
    pm = 1 - pu - pd

    return dt, u, d, m, pu, pm, pd

def trinomial_tree_option_price(S0, K, r, sigma, T, steps):
    dt, u, d, m, pu, pm, pd = calculate_trinomial_parameters(S0, r, sigma, T, steps)

    # Initialize asset prices at maturity
    ST = np.zeros((2 * steps + 1,))
    for i in range(2 * steps + 1):
        ST[i] = S0 * (u ** max(0, steps - i)) * (d ** max(0, i - steps))

    # Option values at maturity for put option
    put_values = np.maximum(K - ST, 0)

    # Step back through the tree
    for j in range(steps - 1, -1, -1):
        for i in range(2 * j + 1):
            put_values[i] = np.exp(-r * dt) * (pu * put_values[i] + pm * put_values[i + 1] + pd * put_values[i + 2])

    return put_values[0]

# Price put options for each strike
option_prices = {}
for moneyness, K in strike_prices.items():
    price = trinomial_tree_option_price(S0, K, r, sigma, T, steps)
    option_prices[moneyness] = price
    print(f"European Put Option Price for {moneyness} (Strike={K}): {round(price, 4)}")

# Display trend in option prices
print("\nTrend in option prices with respect to moneyness:")
for moneyness, price in option_prices.items():
    print(f"{moneyness}: {round(price, 4)}")


European Put Option Price for Deep OTM (Strike=110.00000000000001): 9.8253
European Put Option Price for OTM (Strike=105.0): 6.1776
European Put Option Price for ATM (Strike=100.0): 3.3678
European Put Option Price for ITM (Strike=95.0): 1.5375
European Put Option Price for Deep ITM (Strike=90.0): 0.5537

Trend in option prices with respect to moneyness:
Deep OTM: 9.8253
OTM: 6.1776
ATM: 3.3678
ITM: 1.5375
Deep ITM: 0.5537


**Summary of Findings**

. **Delta and Vega Sensitivities**: We observed that call options tend to have positive Delta, while puts have negative Delta, indicating directional sensitivities. Vega was positive for both, as higher volatility increased the potential for both types of options to reach profitable levels.

. **Trinomial Tree Pricing**: The trinomial model provided a flexible and accurate approach to pricing European options at different moneyness levels, highlighting how option prices increase as the likelihood of finishing in-the-money increases.

. **Practical Insights**: This analysis provided a structured approach to understanding option price sensitivities and trends, showing that options with favorable probabilities of expiring in-the-money command higher prices due to their intrinsic value potential.